In [ ]:
#| default_exp compiling

In [ ]:
#| export
import subprocess
import tensorflow as tf
import os
import pandas as pd
import binascii

2023-05-30 12:11:25.615074: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-30 12:11:25.667018: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-30 12:11:25.667832: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-30 12:11:26.516834: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [ ]:
#| export

def convert_model(train_ds, model_path, model_obj):
    """Model conversion into TFLite model
nbdev
    Args:
        `train_ds` (_dataset_): Training data used for the quantization process
        `model_path`: Path to model files
        `model_obj`: model object, needed for input_shape
    """

    model = f'{model_path}/keras_model'
    
    # Convert the model to the TensorFlow Lite format without quantization
    converter = tf.lite.TFLiteConverter.from_saved_model(model)
    model_no_quant_tflite = converter.convert()
    # Save the model to disk
    open(f'{model_path}/model_no_quant.tflite', "wb").write(model_no_quant_tflite)

    # Convert the model with quantization.
    converter = tf.lite.TFLiteConverter.from_saved_model(model)
    converter.optimizations = [tf.lite.Optimize.DEFAULT]
    
    converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
    converter.inference_input_type = tf.int8
    converter.inference_output_type = tf.int8
    
    input_shape = model_obj.layers[0].input_shape
    
    def representative_dataset():
        for images, labels in train_ds.take(96):
            for img in images:
                input = tf.cast(img, tf.float32)
                input = tf.reshape(input, [1, input_shape[1],input_shape[2]])
                yield([input])

    converter.representative_dataset = representative_dataset
    tflite_model = converter.convert()

    # Save the model.
    with open(f'{model_path}/model.tflite', 'wb') as f:
        f.write(tflite_model)

def convert_to_c_array(bytes)->str:
    """C array conversion"""
    hexstr = binascii.hexlify(bytes).decode("UTF-8") 
    hexstr = hexstr.upper() 
    array = ["0x" + hexstr[i:i + 2] for i in range(0, len(hexstr), 2)] 
    array = [array[i:i+10] for i in range(0, len(array), 10)] 
    return ",\n  ".join([", ".join(e) for e in array])

def convert_model_to_cc(model_path : str):
    """Creates model.cc from model.tflite in folder `model_path`"""
    tflite_binary = open(f'{model_path}/model.tflite', 'rb').read()
    ascii_bytes = convert_to_c_array(tflite_binary)
    header_file = "#include \"person_detect_model_data.h\"\nconst unsigned char model_tflite[] = {\n  " + ascii_bytes + "\n};\nunsigned int model_tflite_len = " + str(len(tflite_binary)) + ";"
    with open(f"{model_path}/model.cc", "w") as f:
        f.write(header_file)

def plot_size(model_path):
    """Plots the size difference before and after quantization
    Args:
        `model_path`: Path to model files
    Returns:
        pandas dataframe: Pandas dataframe containing information
    """

    size_no_quant_tflite = os.path.getsize(f'{model_path}/model_no_quant.tflite')
    size_tflite = os.path.getsize(f'{model_path}/model.tflite')
    
    frame = pd.DataFrame.from_records(
        [["TensorFlow Lite", f"{size_no_quant_tflite} bytes "],
        ["TensorFlow Lite Quantized", f"{size_tflite} bytes", f"{size_no_quant_tflite - size_tflite} bytes"]],
        columns = ["Model", "Size", "Size Reduced"], index="Model")

    return frame
